In [1]:
import arcpy, os, numpy, re
from arcpy import env
import pandas as pd

In [2]:
env.workspace = r"T:\MPO\RTP\FY20 2045 Update\Data and Resources\Network_Analysis\Network_Analysis.gdb"
env.overwriteOutput = True

In [3]:
input_folder = r"T:\MPO\RTP\FY20 2045 Update\Data and Resources"

In [4]:
EFAbound = os.path.join(input_folder, "PerformanceAnalysis", "service_transit_equity", "equity_area.shp")

In [5]:
AOI = "EFA"
service = "Jobs"
travel_mode = "Biking"
year = 2020

In [6]:
layer_name = AOI + service + travel_mode + str(year)

In [7]:
EquityAreaID = pd.read_csv("../EquityAreaID.csv")

In [8]:
EquityAreaID.head()

,BlkGrp10,EquityArea
0,410390019043,5
1,410390020022,6
2,410390026002,10
3,410390033023,18
4,410390043001,29


In [12]:
EquityAreaID['BlkGrp10'].values[0]

410390019043

In [26]:
[i for i in EquityAreaID.index]

[0, 1, 2, 3, 4, 5]

In [14]:
BlkGrp10 = EquityAreaID['BlkGrp10'].values[0]

In [15]:
EFAbound = arcpy.management.SelectLayerByAttribute(EFAbound, "NEW_SELECTION", "BlkGrp10 = '{0}'".format(BlkGrp10), None)

In [16]:
# 2 years * 3 AOI * 2 travel modes * 2 services
sas = ["Jobs", "Amenities"]
travel_modes = ["Biking", "Walking"]
layer_names = ["baseyearJobs_FeatureToPoint", "forecastJobs_FeatureToPoint"]
AOIs = ["MPO", "EFA", "NEFA"]
# matched with layer names
jobfields = ["ojobs", "jobs"]
years = [2020, 2045]
hhfields = ["ohh", "hh"]

In [17]:
newfield = 'weighted_' + jobfields[years.index(year)]

In [19]:
fieldList = arcpy.ListFields(layer_name)
field_names = [f.name for f in fieldList]
if newfield in field_names:
    pass
else:
    arcpy.AddField_management(layer_name, newfield, "FLOAT", "", "", 50)

In [20]:
hhfield = hhfields[years.index(year)]
targetfield = layer_names[years.index(year)] + "_" + jobfields[years.index(year)]

In [21]:
arcpy.management.CalculateField(layer_name, newfield, "!{0}! * !{1}!".format(hhfield, targetfield), "PYTHON3", '', "TEXT")

<Result 'T:\\MPO\\RTP\\FY20 2045 Update\\Data and Resources\\Network_Analysis\\Network_Analysis.gdb\\EFAJobsBiking2020'>

In [22]:
input_layer = arcpy.management.SelectLayerByLocation(layer_name, "COMPLETELY_WITHIN", EFAbound, None, 
                                                                 "NEW_SELECTION", "NOT_INVERT")

In [23]:
HHsum = arcpy.da.TableToNumPyArray(input_layer, hhfield, skip_nulls=True)
WgtJobsSum = arcpy.da.TableToNumPyArray(input_layer, newfield, skip_nulls=True)

In [24]:
round(WgtJobsSum[newfield].sum() / HHsum[hhfield].sum())

8676

In [35]:
table = arcpy.AddJoin_management("SABikingJobs", "FacilityID", "baseyearJobs_FeatureToPoint", "FacilityID", "KEEP_COMMON")
arcpy.CopyFeatures_management(table, "joinedtable")

<Result 'T:\\MPO\\RTP\\FY20 2045 Update\\Data and Resources\\Network_Analysis\\Network_Analysis.gdb\\joinedtable'>

In [4]:
arcpy.management.MakeFeatureLayer("parcels_FeatureToPoint", "baseyear_hh", "ohh > 0")

<Result 'baseyear_hh'>

In [5]:
fieldList = arcpy.ListFields("baseyear_hh")

In [6]:
field_names = [f.name for f in fieldList]

In [7]:
field_names

['OBJECTID',
 'Shape',
 'btype',
 'nrsqft',
 'rsqft',
 'du',
 'yrbuilt',
 'lpid',
 'pundev',
 'dev_land',
 'developed',
 'obtype',
 'orsqft',
 'onrsqft',
 'odu',
 'zid',
 'rezoned',
 'city',
 'annexed',
 'ozid',
 'AreaPerJob',
 'isNonRes',
 'jobs',
 'ojobs',
 'hh',
 'ohh',
 'ORIG_FID',
 'sa_jobs']

In [8]:
newfield = "sa_jobs"

In [9]:
desc = arcpy.Describe("baseyear_hh")

In [10]:
fields = desc.fields

In [11]:
arcpy.ListFields("baseyear_hh", "jobs")[0].type

'Double'

In [12]:
if newfield in field_names:
    pass
else:
    arcpy.AddField_management("baseyear_hh", newfield, "FLOAT", "", "", 20)

In [45]:
service = "Jobs"
travel_mode = "Biking"
AOI = "MPO"
year = 2020

In [36]:
targetHH = arcpy.management.SelectLayerByAttribute("baseyear_hh", "NEW_SELECTION", "ORIG_FID = {0}".format(18778), None)

In [37]:
selectedSA = arcpy.management.SelectLayerByLocation("joinedtable", "COMPLETELY_CONTAINS", targetHH, 
                                               None, "NEW_SELECTION", "NOT_INVERT")

In [38]:
fieldsum = arcpy.da.TableToNumPyArray(selectedSA, "baseyearJobs_FeatureToPoint_jobs", skip_nulls=True)

In [40]:
val = fieldsum["baseyearJobs_FeatureToPoint_jobs"].sum()

In [48]:
val

73822.79370354864

In [44]:
(val*15.44)/127915.574999826

8.910751757824185

In [49]:
arcpy.management.SelectLayerByAttribute("baseyear_hh", "CLEAR_SELECTION")

id,value
0,a Layer object
1,-1


In [54]:
arcpy.analysis.SpatialJoin("baseyearHH_FeatureToPoint", "joinedtable","baseyearjobs_HH","JOIN_ONE_TO_ONE", "KEEP_COMMON",
                           'btype "btype" true true false 80 Text 0 0,First,#,baseyearHH_FeatureToPoint,btype,0,80;nrsqft "nrsqft" true true false 8 Double 0 0,First,#,baseyearHH_FeatureToPoint,nrsqft,-1,-1;rsqft "rsqft" true true false 8 Double 0 0,First,#,baseyearHH_FeatureToPoint,rsqft,-1,-1;du "du" true true false 8 Double 0 0,First,#,baseyearHH_FeatureToPoint,du,-1,-1;yrbuilt "yrbuilt" true true false 8 Double 0 0,First,#,baseyearHH_FeatureToPoint,yrbuilt,-1,-1;lpid "lpid" true true false 8 Double 0 0,First,#,baseyearHH_FeatureToPoint,lpid,-1,-1;pundev "pundev" true true false 8 Double 0 0,First,#,baseyearHH_FeatureToPoint,pundev,-1,-1;dev_land "dev_land" true true false 8 Double 0 0,First,#,baseyearHH_FeatureToPoint,dev_land,-1,-1;developed "developed" true true false 8 Double 0 0,First,#,baseyearHH_FeatureToPoint,developed,-1,-1;obtype "obtype" true true false 80 Text 0 0,First,#,baseyearHH_FeatureToPoint,obtype,0,80;orsqft "orsqft" true true false 8 Double 0 0,First,#,baseyearHH_FeatureToPoint,orsqft,-1,-1;onrsqft "onrsqft" true true false 8 Double 0 0,First,#,baseyearHH_FeatureToPoint,onrsqft,-1,-1;odu "odu" true true false 8 Double 0 0,First,#,baseyearHH_FeatureToPoint,odu,-1,-1;zid "zid" true true false 8 Double 0 0,First,#,baseyearHH_FeatureToPoint,zid,-1,-1;rezoned "rezoned" true true false 8 Double 0 0,First,#,baseyearHH_FeatureToPoint,rezoned,-1,-1;city "city" true true false 80 Text 0 0,First,#,baseyearHH_FeatureToPoint,city,0,80;annexed "annexed" true true false 8 Double 0 0,First,#,baseyearHH_FeatureToPoint,annexed,-1,-1;ozid "ozid" true true false 8 Double 0 0,First,#,baseyearHH_FeatureToPoint,ozid,-1,-1;AreaPerJob "AreaPerJob" true true false 8 Double 0 0,First,#,baseyearHH_FeatureToPoint,AreaPerJob,-1,-1;isNonRes "isNonRes" true true false 4 Long 0 0,First,#,baseyearHH_FeatureToPoint,isNonRes,-1,-1;jobs "jobs" true true false 8 Double 0 0,First,#,baseyearHH_FeatureToPoint,jobs,-1,-1;ojobs "ojobs" true true false 8 Double 0 0,First,#,baseyearHH_FeatureToPoint,ojobs,-1,-1;hh "hh" true true false 8 Double 0 0,First,#,baseyearHH_FeatureToPoint,hh,-1,-1;ohh "ohh" true true false 8 Double 0 0,First,#,baseyearHH_FeatureToPoint,ohh,-1,-1;ORIG_FID "ORIG_FID" true true false 4 Long 0 0,First,#,baseyearHH_FeatureToPoint,ORIG_FID,-1,-1;baseyearJobs_FeatureToPoint_ojobs "ojobs" true true false 8 Double 0 0,Sum,#,joinedtable,baseyearJobs_FeatureToPoint_ojobs,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,joinedtable,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,joinedtable,Shape_Area,-1,-1', 
                           "COMPLETELY_WITHIN", None, '')

<Result 'T:\\MPO\\RTP\\FY20 2045 Update\\Data and Resources\\Network_Analysis\\Network_Analysis.gdb\\baseyearjobs_HH'>

In [55]:
arcpy.analysis.SpatialJoin("baseyearHH_FeatureToPoint", "joinedtable","baseyear_jobs_HH","JOIN_ONE_TO_ONE", "KEEP_COMMON",
                           'btype "btype" true true false 80 Text 0 0,First,#,{0},btype,0,80;nrsqft "nrsqft" true true false 8 Double 0 0,First,#,{0},nrsqft,-1,-1;rsqft "rsqft" true true false 8 Double 0 0,First,#,{0},rsqft,-1,-1;du "du" true true false 8 Double 0 0,First,#,{0},du,-1,-1;yrbuilt "yrbuilt" true true false 8 Double 0 0,First,#,{0},yrbuilt,-1,-1;lpid "lpid" true true false 8 Double 0 0,First,#,{0},lpid,-1,-1;pundev "pundev" true true false 8 Double 0 0,First,#,{0},pundev,-1,-1;dev_land "dev_land" true true false 8 Double 0 0,First,#,{0},dev_land,-1,-1;developed "developed" true true false 8 Double 0 0,First,#,{0},developed,-1,-1;obtype "obtype" true true false 80 Text 0 0,First,#,{0},obtype,0,80;orsqft "orsqft" true true false 8 Double 0 0,First,#,{0},orsqft,-1,-1;onrsqft "onrsqft" true true false 8 Double 0 0,First,#,{0},onrsqft,-1,-1;odu "odu" true true false 8 Double 0 0,First,#,{0},odu,-1,-1;zid "zid" true true false 8 Double 0 0,First,#,{0},zid,-1,-1;rezoned "rezoned" true true false 8 Double 0 0,First,#,{0},rezoned,-1,-1;city "city" true true false 80 Text 0 0,First,#,{0},city,0,80;annexed "annexed" true true false 8 Double 0 0,First,#,{0},annexed,-1,-1;ozid "ozid" true true false 8 Double 0 0,First,#,{0},ozid,-1,-1;AreaPerJob "AreaPerJob" true true false 8 Double 0 0,First,#,{0},AreaPerJob,-1,-1;isNonRes "isNonRes" true true false 4 Long 0 0,First,#,{0},isNonRes,-1,-1;jobs "jobs" true true false 8 Double 0 0,First,#,{0},jobs,-1,-1;ojobs "ojobs" true true false 8 Double 0 0,First,#,{0},ojobs,-1,-1;hh "hh" true true false 8 Double 0 0,First,#,{0},hh,-1,-1;ohh "ohh" true true false 8 Double 0 0,First,#,{0},ohh,-1,-1;ORIG_FID "ORIG_FID" true true false 4 Long 0 0,First,#,{0},ORIG_FID,-1,-1;{1}_{2} "{2}" true true false 8 Double 0 0,Sum,#,joinedtable,{1}_{2},-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,joinedtable,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,joinedtable,Shape_Area,-1,-1'.format('baseyearHH_FeatureToPoint', 'baseyearJobs_FeatureToPoint', 'ojobs'), 
                           "COMPLETELY_WITHIN", None, '')

<Result 'T:\\MPO\\RTP\\FY20 2045 Update\\Data and Resources\\Network_Analysis\\Network_Analysis.gdb\\baseyear_jobs_HH'>